In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

url = 'https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv'
df = pd.read_csv(url, parse_dates=['date']).dropna()
df = df[(df.league=='Barclays Premier League') | (df.league=='Major League Soccer') | (df.league=='Chinese Super League')]
df['spi_margin'] = np.abs(df['spi1'] - df['spi2'])

In [2]:
ohe = OneHotEncoder()
t1 = df['team1'].unique().tolist()
t2 = df['team2'].unique().tolist()
team1 = df.iloc[:,4:5].values
team1 = ohe.fit_transform(team1).toarray()
team1 = pd.DataFrame(team1).reset_index(drop=True)
team1.columns = t1
team2 = df.iloc[:,5:6].values
team2 = ohe.fit_transform(team2).toarray()
team2 = pd.DataFrame(team2).reset_index(drop=True)
team2.columns = t2
X_num = df[['spi1', 'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2', 'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1', 'adj_score2']].reset_index(drop=True)
X = pd.concat([X_num, team1, team2], axis=1)
y = df[['spi_margin']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=6)
y_train = ohe.fit_transform(y_train).toarray()
y_test = ohe.fit_transform(y_test).toarray()

In [3]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(165, )))
model.add(Dense(64, activation='relu', name='dense_2'))
model.add(Dense(1, activation='linear', name='dense_output'))

In [4]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [5]:
model.fit(X_train, y_train, epochs=5, batch_size=128)

Epoch 1/5
21/21 [==============================] - 1s 5ms/step - loss: 8.5982 - mae: 1.8700
Epoch 2/5
21/21 [==============================] - 0s 6ms/step - loss: 0.8490 - mae: 0.7655
Epoch 3/5
21/21 [==============================] - 0s 5ms/step - loss: 1.1779 - mae: 0.9705
Epoch 4/5
21/21 [==============================] - 0s 5ms/step - loss: 0.7499 - mae: 0.7549
Epoch 5/5
21/21 [==============================] - 0s 5ms/step - loss: 0.7534 - mae: 0.7833


In [6]:
mse, mae = model.evaluate(X_test, y_test)

28/28 [==============================] - 0s 1ms/step - loss: 0.7136 - mae: 0.7707


In [30]:
print("Mean Average Error: %.2f" % mae)
print("Mean Y Value: %.2f" % np.mean(df['spi_margin']))
m_calc = mae / np.mean(df['spi_margin'])
print("Mean Accuracy: %.2f" % ((1 - m_calc) * 100) + "%")

Mean Average Error: 0.77
Mean Y Value: 10.63
Mean Accuracy: 92.75%
